# **DeepSparse-Milvus Semantic Search Client**

This notebook demonstrates how to interact with the Semantic Search application over HTTP.

In [4]:
import requests

# if running on localhost
# base_url = "localhost"

# if running on AWS - add your app servers' public IP
base_url = "34.227.11.121"

### **Drop Existing Data**

`/drop` route used to drop existing colleciton if it exists.

In [5]:
# /drop path drops the collection
url = f'http://{base_url}:5000/drop'

response = requests.post(url)
print(response.text)

"Collection does not exist"


### **Load New Data**

`/load` route used to generate embeddings with DeepSparse and load data into Milvus.

The `/load` path accepts a `csv` file with `(title, text)` pairs. When called, this endpoint generates `embeddings` with DeepSparse on each `text` and inserts the `(id, embedding)` in Milvus and the `(id, title, text)`. Since we are generating embeddings for every element in the list, this may take a bit of time.

There is an `example.csv` file available.

In [6]:
url = f'http://{base_url}:5000/load'

response = requests.post(url, files={
    'file': open('example.csv', 'rb')
})
print(response.text)

"Successfully loaded data. Inference Time 4.723909034999906; DB Load Time 3.0950468900000487"


### **Check It Worked**

`/count` route allows you to check the number of elements in the database. You should see 160 if you used the `example.csv` file.

In [7]:
url = f'http://{base_url}:5000/count'

response = requests.post(url)
print(response.text)

160


### **Search The Database**

`/search` endpoint accepts a sentence over `GET`. Let's try 50 uploads to see what the responses and latency looks like.

In [8]:
sentence = "The United States has brokered a  cease-fire between a renegade Afghan militia leader and the  embattled governor of the western province of Herat,  Washington's envoy to Kabul said Tuesday."
print(sentence)

The United States has brokered a  cease-fire between a renegade Afghan militia leader and the  embattled governor of the western province of Herat,  Washington's envoy to Kabul said Tuesday.


In [13]:
import json
for _ in range(50):
    query_str = "query_sentence=" + sentence
    url = f'http://{base_url}:5000/search'
    response = json.loads(requests.get(url, query_str).text)

We can see that the responses include articles that are similiar to the baseline text.

In [14]:
for idx in response:
    print(response[idx]['title'])

U.S. Brokers Cease-fire in Western Afghanistan
Afghan Army Dispatched to Calm Violence
Delegates Urge Cleric to Pull Out of Najaf
Delegation Is Delayed Before Reaching Najaf
Karzai Promises Afghans Security for Election (Reuters)
Georgian president calls for international conference on South Ossetia
Fresh Fighting Shatters Short-Lived Ceasefire Deal
Iran Warns Its Missiles Can Hit Anywhere in Israel
Peace delegation leaves Najaf empty-handed as fighting continues


### **Check Latency**

`/latency` endpoint checks the latency of recent calls. Calling the endpoint also clears old latency data.

In [15]:
url = f'http://{base_url}:5000/latency'

response = requests.post(url)
print(json.loads(response.text))

{'count': 50, 'median': 99.72352400006912, 'mean': 99.62290793999728, 'std': 0.7136873781060545}
